In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import interval_bound_propagation as ibp
import sys
sys.path.append('../')
from output_utils import *
sys.path.append('../../')
from utils import *
train_data, train_label, test_data, test_label = YALE_split('../../datasets/yale/YALEBXF.mat')

os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='6' 
sess= tf.InteractiveSession()

/home/liweis/.conda/envs/tf12_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/liweis/.conda/envs/tf12_py3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/home/liweis/.conda/envs/tf12_py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/liweis/.conda/envs/tf12_py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def Yale_model(width_num=1):
    return (
            ('conv2d', (3, 3), 4*width_num, 'SAME', 1),
            ('activation', 'relu'),
            ('conv2d', (3, 3), 4*width_num, 'SAME', 2),
            ('activation', 'relu'),
            ('conv2d', (3, 3), 8*width_num, 'SAME', 1),
            ('activation', 'relu'),
            ('conv2d', (3, 3), 8*width_num, 'SAME', 2),
            ('activation', 'relu'),
            ('conv2d', (3, 3), 16*width_num, 'SAME', 1),
            ('activation', 'relu'),
            ('conv2d', (3, 3), 16*width_num, 'SAME', 2),
            ('activation', 'relu'),
            ('conv2d', (3, 3), 32*width_num, 'SAME', 1),
            ('activation', 'relu'),
            ('conv2d', (3, 3), 32*width_num, 'SAME', 2),
            ('activation', 'relu'),
            ('linear', 200),
            ('activation', 'relu'))

In [3]:
data_train = (train_data, train_label)
data_test = (test_data, test_label)
data = ibp.build_dataset(data_train, batch_size=20,sequential=False)
num_classes = len(set(test_label))
predictor = ibp.DNN(num_classes, Yale_model(width_num=2), 0.0002)
predictor = ibp.VerifiableModelWrapper(predictor)

# Training.
train_losses, train_loss, _ = ibp.create_classification_losses(
  20001,
  data.image,
  data.label,
  predictor,
  0.5,
  loss_weights={
      'nominal': {'init': 1.0,
                  'final': 0.5},
      'attack': {'init': 0.0,
                 'final': 0.5},
      'verified': {'init': 0.0,
                   'final': 0.0},
  },
  warmup_steps=500,
  rampup_steps=6500,
  input_bounds=(0., 1.))

saver = tf.train.Saver(predictor.wrapped_network.get_variables())


/home/liweis/.conda/envs/tf12_py3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/data/scsi/home/liweis/Adv_ML/Github_repos/privacy-vs-robustness/interval bound propagation-based verification/Yale/interval_bound_propagation/src/model.py:146: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn('Unprocessed module "%s"', str(m))
W0717 21:12:11.576085 140537997150016 model.py:146] Unprocessed module "<interval_bound_propagation.src.model.DNN object at 0x7fd17c4489b0>"


In [4]:
x_placeholder = tf.placeholder(shape=(None, 192, 168, 1), dtype=tf.float32)
y_placeholder = tf.placeholder(shape=(None), dtype=tf.int64)
model_logit = predictor.wrapped_network(x_placeholder)

saver.restore(sess,os.path.join('./model_robust','checkpoint'))
num_step = 20
max_perturb = 8/255
batch_size = 500
step_size = 2.5*max_perturb/num_step

output_train_benign, output_train_adversarial, output_test_benign, output_test_adversarial = \
classifier_performance(sess, model_logit, train_data, train_label, x_placeholder, y_placeholder,
                       test_data, test_label, batch_size, num_step, max_perturb, step_size)

results_dir = './output_results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
np.savez(os.path.join(results_dir, 'model_robust.npz'), 
         output_train_benign = output_train_benign, output_test_benign = output_test_benign,
         output_train_adversarial = output_train_adversarial, output_test_adversarial = output_test_adversarial,
         train_label = train_label, test_label = test_label)

INFO:tensorflow:Restoring parameters from ./model_robust/checkpoint


I0717 21:12:12.145470 140537997150016 tf_logging.py:115] Restoring parameters from ./model_robust/checkpoint


Benign accuracy:  (0.991578947368421, 0.857976653696498)  Adversarial accuracy:  (0.9442105263157895, 0.6867704280155642)
